# Deprecated

In [28]:
import json
import requests
import urllib
import requests
from json import loads
import pandas as pd

In [29]:
import os
from dotenv import load_dotenv

load_dotenv()
X_ELS_APIKey = os.getenv("X_ELS_APIKey")
X_ELS_Insttoken = os.getenv("X_ELS_Insttoken")

In [ ]:
keyword = "%28cancer+of+the+prostate%29+%28+molecular+targeted+therapy%29"

## Scopus

Pontos que precisaremos trabalhar -

- aceita no máximo 25 artigos por query
- https://dev.elsevier.com/documentation/ScopusSearchAPI.wadl
- Precisamos usar uma outra chave de api disponibilizada pelo teste online usando CORS 
https://dev.elsevier.com/search.html#/Scopus_Search

In [33]:
url = f"http://api.elsevier.com/content/search/scopus?query=KEY{keyword}"

# Make a request for a given paper
resp = requests.get(
    url,
    headers={
        "Accept": "application/json",
        "X-ELS-APIKey": X_ELS_APIKey,
        "X-ELS-Insttoken": X_ELS_Insttoken,
        "X-RateLimit-Reset": None,
    },
)

# Check the status of the request
print(resp.status_code)

# Check if the problem is the weekly download limit (usually 10000)
try:
    print(resp.header["X-RateLimit-Remaining"])
except:
    pass

200


In [ ]:
dic = loads(resp.content)

data = pd.DataFrame.from_dict(dic)

for i in data.iloc[0]:
    scopus_data = pd.DataFrame(i)

scopus_data.head()

## Science Direct

Pontos que precisaremos trabalhar
- aceita no máximo 100 artigos por query
- https://dev.elsevier.com/documentation/ScienceDirectSearchAPI.wadl
- Precisamos usar uma outra chave de api disponibilizada pelo teste online usanod CORS 
https://dev.elsevier.com/search.html#/ScienceDirect_Search_V2

In [ ]:
url = f"https://api.elsevier.com/content/search/sciencedirect?query={keyword}&count=100"

# Make a request for a given paper
resp = requests.get(
    url,
    headers={
        "Accept": "application/json",
        "X-ELS-APIKey": X_ELS_APIKey,
        "X-ELS-Insttoken": X_ELS_Insttoken,
        "X-RateLimit-Reset": None,
    },
)

# Check the status of the request
print(resp.status_code)

# Check if the problem is the weekly download limit (usually 10000)
try:
    print(resp.header["X-RateLimit-Remaining"])
except:
    pass

In [ ]:
dic = loads(resp.content)

data = pd.DataFrame.from_dict(dic)

for i in data.iloc[0]:
    scidir_data = pd.DataFrame(i)

scidir_data.columns

# Version 2

In [2]:
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import json
import pandas as pd

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
apikey = os.getenv("X_ELS_APIKey")
insttoken = os.getenv("X_ELS_Insttoken")
keyword = "(cancer of the prostate) AND (molecular targeted therapy)"

In [5]:
client = ElsClient(apikey)
client.inst_token = insttoken

## Science Direct

In [16]:
doc_srch = ElsSearch("cancer prostata", "sciencedirect")
t = doc_srch.execute(client, get_all=False)
print("doc_srch has", len(doc_srch.results), "results.")

doc_srch has 25 results.


In [17]:
abstract = []
pubtype = []

for i in doc_srch.results_df["prism:doi"]:
    doi_doc = FullDoc(doi=i)
    if doi_doc.read(client):
        abstract.append(doi_doc.data["coredata"]["dc:description"])
        pubtype.append(doi_doc.data["coredata"]["pubType"])
        # doi_doc.write()
    else:
        print("Read document failed.")

doc_srch.results_df["abstract"] = abstract
doc_srch.results_df["pubtype"] = pubtype

In [18]:
# A ideia é que retorne um dataset onde cada linha é um artigo
doc_srch.results_df

doc_srch.results_df.to_csv("../../data/sci_dir_data.csv")

,@_fa,load-date,link,dc:identifier,prism:url,dc:title,dc:creator,prism:publicationName,prism:coverDate,prism:doi,openaccess,pii,authors,prism:volume,prism:startingPage,prism:endingPage,abstract,pubtype
0,true,2023-02-03T00:00:00.000Z,{'self': 'https://api.elsevier.com/content/art...,DOI:10.1016/j.rx.2022.12.005,https://api.elsevier.com/content/article/pii/S...,"Resonancia magnética en la detección, estadifi...",J. C. Vilanova,Radiología,2023-02-03,10.1016/j.rx.2022.12.005,False,S0033833823000097,"{'author': [{'$': 'J. C. Vilanova'}, {'$': 'V....",NaN,NaN,NaN,\n El cáncer de próstata es m...,fla
1,true,2023-03-24T00:00:00.000Z,{'self': 'https://api.elsevier.com/content/art...,DOI:10.1016/j.acuro.2023.03.003,https://api.elsevier.com/content/article/pii/S...,Impacto de la realización de resonancia magnét...,D. Vieira e Brito,Actas Urológicas Españolas,2023-03-24,10.1016/j.acuro.2023.03.003,False,S0210480623000542,"{'author': [{'$': 'D. Vieira e Brito'}, {'$': ...",NaN,NaN,NaN,\n Introducción\n ...,fla
2,true,2022-10-30T00:00:00.000Z,{'self': 'https://api.elsevier.com/content/art...,DOI:10.1016/S1761-3310(22)47080-0,https://api.elsevier.com/content/article/pii/S...,"Técnicas, indicaciones y resultados del vaciam...",C. Laine,EMC - Urología,2022-12-31,10.1016/S1761-3310(22)47080-0,False,S1761331022470800,"{'author': [{'$': 'C. Laine'}, {'$': 'F. Rozet...",54,1,6,\n El vaciamiento ganglionar ...,rev


## Scopus

In [7]:
doc_srch_scopus = ElsSearch("cancer prostata", "scopus")
# Get all é o campo que dizemos se queremos ou não todos os artigos
t = doc_srch_scopus.execute(client, get_all=False)
print("doc_srch has", len(doc_srch_scopus.results), "results.")

doc_srch has 25 results.


In [8]:
doc_srch_scopus.results_df.head(3)

,@_fa,link,prism:url,dc:identifier,eid,dc:title,dc:creator,prism:publicationName,prism:eIssn,prism:volume,...,subtype,subtypeDescription,article-number,source-id,openaccess,openaccessFlag,freetoread,freetoreadLabel,prism:issn,pii
0,true,{'self': 'https://api.elsevier.com/content/abs...,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85156209969,2-s2.0-85156209969,Efficacy and safety of Water Vapor Thermal The...,Yang J.,BMC Urology,14712490,23,...,ar,Article,72,19397,1,True,"{'value': [{'$': 'all'}, {'$': 'publisherfullg...","{'value': [{'$': 'All Open Access'}, {'$': 'Go...",NaN,NaN
1,true,{'self': 'https://api.elsevier.com/content/abs...,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85150848143,2-s2.0-85150848143,Comparing the effectiveness and lubricity of a...,Adanu K.K.,Scientific Reports,20452322,13,...,ar,Article,4666,21100200805,1,True,"{'value': [{'$': 'all'}, {'$': 'publisherfullg...","{'value': [{'$': 'All Open Access'}, {'$': 'Go...",NaN,NaN
2,true,{'self': 'https://api.elsevier.com/content/abs...,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85149596265,2-s2.0-85149596265,Influence of benign prostatic hyperplasia patt...,Boschheidgen M.,CVIR Endovascular,25208934,6,...,ar,Article,9,21101023028,1,True,"{'value': [{'$': 'all'}, {'$': 'publisherfullg...","{'value': [{'$': 'All Open Access'}, {'$': 'Go...",NaN,NaN


In [17]:
doc_srch_scopus.results_df.to_csv("../../data/scopus_data.csv")

### Hoje (15/05) essa função que pega os resumos dos artigos não está funcionando 
O erro está ocorrendo devido o limite de requisição excedido, por semana 
precisamos trabalhar nesse ponto 

In [9]:
dicts = {}

for i in doc_srch_scopus.results_df["prism:url"]:
    scp_doc = AbsDoc(uri=i)
    if scp_doc.read(client):
        if "dc:description" in scp_doc.data["coredata"]:
            dicts[i] = scp_doc.data["coredata"]["dc:description"]
        else:
            dicts[i] = "None"
    else:
        dicts[i] = "Failed"

In [17]:
abstracts_df = pd.DataFrame(dicts.items(), columns=["prism:url", "Abstract"])
doc_srch_scopus.results_df.merge(abstracts_df, on="prism:url", how="left")
doc_srch_scopus.results_df

,@_fa,link,prism:url,dc:identifier,eid,dc:title,dc:creator,prism:publicationName,prism:eIssn,prism:volume,...,subtype,subtypeDescription,article-number,source-id,openaccess,openaccessFlag,freetoread,freetoreadLabel,prism:issn,pii
0,true,{'self': 'https://api.elsevier.com/content/abs...,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85156209969,2-s2.0-85156209969,Efficacy and safety of Water Vapor Thermal The...,Yang J.,BMC Urology,14712490,23,...,ar,Article,72,19397,1,True,"{'value': [{'$': 'all'}, {'$': 'publisherfullg...","{'value': [{'$': 'All Open Access'}, {'$': 'Go...",NaN,NaN
1,true,{'self': 'https://api.elsevier.com/content/abs...,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85150848143,2-s2.0-85150848143,Comparing the effectiveness and lubricity of a...,Adanu K.K.,Scientific Reports,20452322,13,...,ar,Article,4666,21100200805,1,True,"{'value': [{'$': 'all'}, {'$': 'publisherfullg...","{'value': [{'$': 'All Open Access'}, {'$': 'Go...",NaN,NaN
2,true,{'self': 'https://api.elsevier.com/content/abs...,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85149596265,2-s2.0-85149596265,Influence of benign prostatic hyperplasia patt...,Boschheidgen M.,CVIR Endovascular,25208934,6,...,ar,Article,9,21101023028,1,True,"{'value': [{'$': 'all'}, {'$': 'publisherfullg...","{'value': [{'$': 'All Open Access'}, {'$': 'Go...",NaN,NaN
3,true,{'self': 'https://api.elsevier.com/content/abs...,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85149222913,2-s2.0-85149222913,Excess healthcare costs of mental disorders in...,Larrañaga I.,Cost Effectiveness and Resource Allocation,14787547,21,...,ar,Article,18,12788,1,True,"{'value': [{'$': 'all'}, {'$': 'publisherfullg...","{'value': [{'$': 'All Open Access'}, {'$': 'Go...",NaN,NaN
4,true,{'self': 'https://api.elsevier.com/content/abs...,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85148551386,2-s2.0-85148551386,Economic burden of locoregional and metastatic...,De Castro J.,BMC Pulmonary Medicine,14712466,23,...,ar,Article,69,18026,1,True,NaN,NaN,NaN,NaN
5,true,{'self': 'https://api.elsevier.com/content/abs...,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85147591995,2-s2.0-85147591995,Characterization of the population affiliated ...,Mora-Moreo L.,International Journal for Equity in Health,14759276,22,...,re,Review,28,21461,1,True,"{'value': [{'$': 'all'}, {'$': 'publisherfullg...","{'value': [{'$': 'All Open Access'}, {'$': 'Go...",NaN,NaN
6,true,{'self': 'https://api.elsevier.com/content/abs...,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85145527037,2-s2.0-85145527037,Prognostic significance of pretreatment system...,Zhang B.,World Journal of Surgical Oncology,14777819,21,...,ar,Article,2,13231,1,True,"{'value': [{'$': 'all'}, {'$': 'publisherfullg...","{'value': [{'$': 'All Open Access'}, {'$': 'Go...",NaN,NaN
7,true,{'self': 'https://api.elsevier.com/content/abs...,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85145492354,2-s2.0-85145492354,Hyaluronic acid spacer in prostate cancer radi...,Björeland U.,Radiation Oncology,1748717X,18,...,ar,Article,1,5800179603,1,True,"{'value': [{'$': 'all'}, {'$': 'publisherfullg...","{'value': [{'$': 'All Open Access'}, {'$': 'Go...",NaN,NaN
8,true,{'self': 'https://api.elsevier.com/content/abs...,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85152428317,2-s2.0-85152428317,Regulatory mechanism of DHRS2-modified human u...,Wu X.,Tissue and Cell,15323072,82,...,ar,Article,102078,19057,0,False,NaN,NaN,00408166,S0040816623000666
9,true,{'self': 'https://api.elsevier.com/content/abs...,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85151435775,2-s2.0-85151435775,Synergistic anti-cancer and attenuation effect...,Wang S.,Biomedicine and Pharmacotherapy,19506007,162,...,ar,Article,114633,28620,1,True,NaN,NaN,07533322,S0753332223004213


## Primeira forma testada - **Não considerar** 

In [52]:
abstract_scopus = []
for i in doc_srch_scopus.results_df["prism:url"]:
    scp_doc = AbsDoc(uri=i)
    if scp_doc.read(client):
        if "dc:description" in scp_doc.data["coredata"]:
            abstract_scopus.append(scp_doc.data["coredata"]["dc:description"])
        else:
            abstract_scopus.append("None")
    else:
        print("Read document failed.")

Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.
Read document failed.


## TESTE - Como pegar os resumos

In [ ]:
# science direct
doi_doc = FullDoc(doi="10.1016/j.acuro.2023.03.011")
if doi_doc.read(client):
    print("doi_doc.title: ", doi_doc.title)
    doi_doc.write()
else:
    print("Read document failed.")

In [ ]:
doi_doc.data

In [34]:
# Scopus - não está funcionando
scp_doc = AbsDoc(scp_id=84872135457)
if scp_doc.read(client):
    print("scp_doc.title: ", scp_doc.title)
    scp_doc.write()
else:
    print("Read document failed.")

Read document failed.


In [48]:
url = "https://api.elsevier.com/content/abstract/scopus_id/85040730407"
response = requests.get(
    url,
    headers={
        "Accept": "application/json",
        "X-ELS-APIKey": X_ELS_APIKey,
        "X-ELS-Insttoken": X_ELS_Insttoken,
        "X-RateLimit-Reset": None,
        "X-RateLimit-Limit": "200000",
        "X-RateLimit-Remaining": "19636",
    },
)
print(response.headers)

{'Date': 'Mon, 15 May 2023 15:23:27 GMT', 'Content-Type': 'application/json', 'Content-Length': '225', 'Connection': 'keep-alive', 'X-ELS-ReqId': 'd885078f1f853bd6', 'X-ELS-TransId': 'b9b58260c43b1561', 'X-RateLimit-Limit': '10000', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1684627200', 'CF-Cache-Status': 'DYNAMIC', 'Server': 'cloudflare', 'CF-RAY': '7c7c61d5afada68e-GRU'}


In [49]:
response.content

b'{"error-response":{"error-code":"TOO_MANY_REQUESTS","error-message":"Request has been placed in time-out for exceeding quota or rate limits. Please reference HTTP header X-RateLimit-Reset for when requests can resubmitted."}}'